In [27]:
import os
import pymssql
import spacy

import pandas as pd
import numpy as np
import locale

from dotenv import load_dotenv
from utils_ccd import get_connection
from docxtpl import DocxTemplate


load_dotenv()
conn = get_connection()

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

from antecedentes import create_antecedentes_doc

import pypdf
from pathlib import Path

In [28]:
!python -m spacy download pt_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 17.5 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [21]:
def get_file_path(row):
    return Path('/media/informacoes_pdf/') / row['setor'].strip() / row['arquivo']

def get_pdf_text(row):
    arquivo = get_file_path(row)
    print(f'File {arquivo} to pypdf text')
    try:
        pdf = pypdf.PdfReader(arquivo)
        text = []
        for page in pdf.pages:
            text.append(page.extract_text())
        return ' '.join(text)
    except FileNotFoundError:
        print(f'File not found: {arquivo}')
        return ''

In [15]:
antecedentes_sql = ''' 

SELECT DISTINCT pro.numero_processo,
       pro.ano_processo,
       pro.assunto,
       pro.interessado,
       pm.Descricao as marcador      
       
FROM processo.dbo.Processos pro
INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp ON pmp.IdProcesso = pro.IdProcesso
INNER JOIN processo.dbo.Pro_Marcador pm ON pmp.IdMarcador = pm.IdMarcador
LEFT JOIN processo.dbo.Exe_Debito ed ON ed.IdProcessoExecucao = pro.IdProcesso
LEFT JOIN processo.dbo.Exe_DebitoPessoa edp ON edp.IDDebito = ed.IdDebito
LEFT JOIN processo.dbo.GenPessoa gp ON gp.IdPessoa = edp.IDPessoa
WHERE pro.setor_atual = 'CCD'
AND lower(pm.Descricao) like '%antecedente%'
'''
processos_antecedentes = pd.read_sql(antecedentes_sql, conn)

/tmp/ipykernel_59294/146702693.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  processos_antecedentes = pd.read_sql(antecedentes_sql, conn)


In [16]:
processos_antecedentes

,numero_processo,ano_processo,assunto,interessado,marcador
0,000761,2022,APURAÇÃO DE RESPONSABILIDADE EM ATENDIMENTO AO...,PREFEITURA MUNICIPAL DE CANGUARETAMA ...,Antecedentes
1,000913,2023,COMUNICAÇÃO DE IRREGULARIDADE ...,SIGILOSO ...,Antecedentes
2,001749,2019,"CONTAS DO CHEFE DO PODER EXECUTIVO MUNICIPAL, ...",PREFEITURA MUNICIPAL DE VIÇOSA ...,Antecedentes
3,003277,2020,AUMENTO REMUNERATÓRIO DURANTE A PANDEMIA - REM...,CÂMARA MUNICIPAL DE ANTÔNIO MARTINS ...,Antecedentes
4,003980,2022,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO PRO...",PREFEITURA MUNICIPAL DE CEARÁ-MIRIM ...,Antecedentes
5,004072,2021,MONITORAMENTO ...,AGÊNCIA DE FOMENTO DO RN S/A ...,Antecedentes
6,005181,2020,AUDITORIA DE CONFORMIDADE-EXERCÍCIO 2018 ...,PREFEITURA MUNICIPAL DE MOSSORÓ ...,Antecedentes
7,006621,2015,CONTAS DO CHEFE DO PODER EXECUTIVO REFERENTE A...,PREF.MUN.UPANEMA ...,Antecedentes


In [17]:
sql_info_antecedentes_gaana = ''' 
WITH RankedData AS (
    SELECT 
    	ai.setor,
        concat(rtrim(ai.setor),'_',
        ai.numero_processo ,'_',
        ai.ano_processo,'_',RIGHT(concat('0000',ai.ordem),4),'.pdf') as arquivo,
        ai.datapublicacao, 
        ppe.SequencialProcessoEvento, 
        ai.numero_processo, 
        ai.ano_processo,
        ROW_NUMBER() OVER (
            PARTITION BY CONCAT(ai.numero_processo, '/', ai.ano_processo)
            ORDER BY ppe.SequencialProcessoEvento DESC
        ) AS rn
    FROM processo.dbo.Ata_Informacao ai 
    LEFT JOIN processo.dbo.Pro_ProcessoEvento ppe 
        ON ai.idInformacao = ppe.IdInformacao
    WHERE CONCAT(ai.numero_processo, '/', ai.ano_processo) IN (
        SELECT CONCAT(pro.numero_processo, '/', pro.ano_processo)
        FROM processo.dbo.Processos pro
        INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp 
            ON pmp.IdProcesso = pro.IdProcesso
        INNER JOIN processo.dbo.Pro_Marcador pm 
            ON pmp.IdMarcador = pm.IdMarcador
        LEFT JOIN processo.dbo.Exe_Debito ed 
            ON ed.IdProcessoExecucao = pro.IdProcesso
        LEFT JOIN processo.dbo.Exe_DebitoPessoa edp 
            ON edp.IDDebito = ed.IdDebito
        LEFT JOIN processo.dbo.GenPessoa gp 
            ON gp.IdPessoa = edp.IDPessoa
        WHERE pro.setor_atual = 'CCD'
        AND lower(pm.Descricao) LIKE '%antecedente%'
    )
    AND ai.setor LIKE '%GAANA%'
)
SELECT * 
FROM RankedData 
WHERE rn = 1
ORDER BY numero_processo, ano_processo;


'''
df_info_gaana = pd.read_sql(sql_info_antecedentes_gaana, conn)

/tmp/ipykernel_59294/14474292.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_info_gaana = pd.read_sql(sql_info_antecedentes_gaana, conn)


In [10]:
df_info_gaana

,setor,arquivo,datapublicacao,SequencialProcessoEvento,numero_processo,ano_processo,rn
0,GAANA,GAANA_000761_2022_0028.pdf,2025-02-07 08:51:56.317,51,000761,2022,1
1,GAANA,GAANA_000913_2023_0027.pdf,2025-02-17 12:20:47.240,46,000913,2023,1
2,GAANA,GAANA_001749_2019_0042.pdf,2025-02-17 12:29:42.660,80,001749,2019,1
3,GAANA,GAANA_003277_2020_0124.pdf,2025-02-14 09:31:09.193,201,003277,2020,1
4,GAANA,GAANA_003980_2022_0029.pdf,2025-01-28 12:49:37.590,42,003980,2022,1
5,GAANA,GAANA_004072_2021_0039.pdf,2025-02-20 13:06:07.243,74,004072,2021,1
6,GAANA,GAANA_005181_2020_0084.pdf,2025-02-18 08:39:55.740,110,005181,2020,1
7,GAANA,GAANA_006621_2015_0037.pdf,2025-01-14 10:15:59.170,73,006621,2015,1


In [22]:
df_info_gaana['texto_pdf'] = df_info_gaana.apply(get_pdf_text, axis=1)

File /media/informacoes_pdf/GAANA/GAANA_000761_2022_0028.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_000913_2023_0027.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_001749_2019_0042.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_003277_2020_0124.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_003980_2022_0029.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_004072_2021_0039.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_005181_2020_0084.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_006621_2015_0037.pdf to pypdf text


In [25]:
print(df_info_gaana.texto_pdf[0])

             
 
1 
 
 
Processo nº: 761/2022 - TC  
Assunto: APURAÇÃO DE RESPONSABILIDADE  
Unidade contábil: PREFEITURA MUNICIPAL DE CANGUARETAMA 
Responsável: Sra. MARIA DE FÁTIMA BORGES MARINHO  
Patrono habilitado nos autos: ADVOGADO FELIPE AUGUSTO 
CORTEZ MEIRA DE MEDEIROS - OAB/RN 3640 
 
 
 
 
 
D E S P A C H O 
 
De ordem da Exma. Sra. Conselheira Substituta Ana 
Paula de Oliveira Gomes,  remeto o feito à DIRETORIA DE 
INSTRUÇÃO PROCESSUAL E CONTROLE DE DECISÕES (DIP), a fim de 
que informe se há condenações transitadas em julgado em 
desfavor da Sra. MARIA DE FÁTIMA BORGES MARINHO , no âmbito 
do TCE/RN. Em caso positivo, informar os tipos de 
condenações, valores condenatórios envolvidos 
(ressarcimentos e multas) e as datas dos respectivos 
trânsitos em julgado. 
 
 
 
 
Sara Kalline da Silva 
Mat. 9.780-2  
 
 
 
 
 
 
 
 
 
 
